# Saved Model Analysis

This notebook is for analyzing a previously trained and saved FlowTransformer model.

In [1]:
import os
import json
import pandas as pd
import tensorflow as tf
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.metrics import classification_report, confusion_matrix

# We need to import the components to reconstruct the FlowTransformer object
from framework.dataset_specification import NamedDatasetSpecifications
from framework.enumerations import EvaluationDatasetSampling
from framework.flow_transformer import FlowTransformer
from framework.flow_transformer_parameters import FlowTransformerParameters
from implementations.classification_heads import *
from implementations.input_encodings import *
from implementations.pre_processings import StandardPreProcessing
from implementations.transformers.basic_transformers import *
from implementations.transformers.named_transformers import *

2025-07-22 15:47:22.918308: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1753199242.929868 1225262 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1753199242.933207 1225262 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2025-07-22 15:47:22.947524: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: SSE4.1 SSE4.2 AVX AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.


## 1. Select and Load Model

List available models from the `saved_models` directory and load the selected model and its configuration.

In [2]:
models_dir = "saved_models"
model_files = [f for f in os.listdir(models_dir) if f.endswith(".keras")]

if not model_files:
    raise FileNotFoundError("No saved models found in the 'saved_models' directory.")

print("Available models:")
for i, model_file in enumerate(model_files):
    print(f"  {i}: {model_file}")

# Select the model to analyze (e.g., the first one)
selected_model_index = 0
selected_model_file = model_files[selected_model_index]
model_name = os.path.splitext(selected_model_file)[0]

model_path = os.path.join(models_dir, selected_model_file)
config_path = os.path.join(models_dir, f"{model_name}_config.json")

print(f"\nSelected model: {model_name}")

# Load model configuration
with open(config_path, 'r') as f:
    config = json.load(f)

print("\nModel Configuration:")
print(json.dumps(config, indent=2))

# The from_config patch is no longer needed since the source file was corrected.

# Load the Keras model with the custom object and safe_mode=False to allow Lambda layer deserialization.
print("Loading model with custom TransformerEncoderBlock...")
model = tf.keras.models.load_model(
    model_path,
    custom_objects={'TransformerEncoderBlock': TransformerEncoderBlock},
    safe_mode=False
)
print("Model loaded successfully.")
model.summary()

Available models:
  0: FlowTransformer_BERT_CSE_CIC_IDS_ws8_bs128_20250722_143415.keras

Selected model: FlowTransformer_BERT_CSE_CIC_IDS_ws8_bs128_20250722_143415

Model Configuration:
{
  "model_name": "FlowTransformer_BERT_CSE_CIC_IDS_ws8_bs128_20250722_143415",
  "timestamp": "20250722_143415",
  "model_format": "native_keras",
  "dataset": {
    "name": "CSE_CIC_IDS",
    "path": "/home/joeldan/dvcon_model/FlowTransformer_Pytorch_Imp/datasets.csv",
    "eval_percent": 0.01,
    "eval_method": "LastRows"
  },
  "model_config": {
    "input_encoding": "NoInputEncoder",
    "sequential_model": "BasicTransformer",
    "classification_head": "LastTokenClassificationHead",
    "window_size": 8,
    "mlp_layer_sizes": [
      128
    ],
    "mlp_dropout": 0.1
  },
  "training_config": {
    "batch_size": 128,
    "epochs": 5,
    "steps_per_epoch": 64,
    "early_stopping_patience": 5,
    "final_epoch": 4
  },
  "optimizer": "adam",
  "loss": "binary_crossentropy",
  "metrics": [
    "b

/home/joeldan/dvcon_model/FlowTransformer_Pytorch_Imp/implementations/transformers/basic/encoder_block.py:93: UserWarning: Typically inner_dimension should be greater than or equal to the input_dimension!
  warnings.warn(f"Typically inner_dimension should be greater than or equal to the input_dimension!")
I0000 00:00:1753199245.391616 1225262 gpu_device.cc:2022] Created device /job:localhost/replica:0/task:0/device:GPU:0 with 3539 MB memory:  -> device: 0, name: NVIDIA GeForce RTX 4050 Laptop GPU, pci bus id: 0000:01:00.0, compute capability: 8.9
/home/joeldan/miniconda3/envs/flowtransformer/lib/python3.10/site-packages/keras/src/layers/layer.py:391: UserWarning: `build()` was called on layer 'block_0_transformer_encoder', however the layer does not have a `build()` method implemented and it looks like it has unbuilt state. This will cause the layer to be marked as built, despite not being actually built, which may cause failures down the line. Make sure to implement a proper `build()`

Model loaded successfully.


Model: "functional"

┏━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┓
┃ Layer (type)        ┃ Output Shape      ┃    Param # ┃ Connected to      ┃
┡━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━┩
│ input_MAX_IP_PKT_L… │ (None, 8, 1)      │          0 │ -                 │
│ (InputLayer)        │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ input_NUM_PKTS_128… │ (None, 8, 1)      │          0 │ -                 │
│ (InputLayer)        │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ input_NUM_PKTS_256… │ (None, 8, 1)      │          0 │ -                 │
│ (InputLayer)        │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ input_RETRANSMITTE… │ (None, 8, 1)      │          0 │ -                 │
│ (InputLayer)        │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ input_LONGEST_FLOW… │ (None, 8, 1)      │          0 │ -                 │
│ (InputLayer)        │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ input_TCP_WIN_MAX_… │ (None, 8, 1)      │          0 │ -                 │
│ (InputLayer)        │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ input_NUM_PKTS_102… │ (None, 8, 1)      │          0 │ -                 │
│ (InputLayer)        │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ input_DURATION_OUT  │ (None, 8, 1)      │          0 │ -                 │
│ (InputLayer)        │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ input_IN_BYTES      │ (None, 8, 1)      │          0 │ -                 │
│ (InputLayer)        │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ input_RETRANSMITTE… │ (None, 8, 1)      │          0 │ -                 │
│ (InputLayer)        │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ input_TCP_WIN_MAX_… │ (None, 8, 1)      │          0 │ -                 │
│ (InputLayer)        │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ input_RETRANSMITTE… │ (None, 8, 1)      │          0 │ -                 │
│ (InputLayer)        │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ input_DURATION_IN   │ (None, 8, 1)      │          0 │ -                 │
│ (InputLayer)        │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ input_SRC_TO_DST_S… │ (None, 8, 1)      │          0 │ -                 │
│ (InputLayer)        │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ input_RETRANSMITTE… │ (None, 8, 1)      │          0 │ -                 │
│ (InputLayer)        │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ input_FLOW_DURATIO… │ (None, 8, 1)      │          0 │ -                 │
│ (InputLayer)        │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ input_MIN_IP_PKT_L… │ (None, 8, 1)      │          0 │ -               

 Total params: 2,347,049 (8.95 MB)

 Trainable params: 782,349 (2.98 MB)

 Non-trainable params: 0 (0.00 B)

 Optimizer params: 1,564,700 (5.97 MB)

## 2. Recreate FlowTransformer and Load Dataset

To properly evaluate the model, we need to recreate the `FlowTransformer` instance with the same configuration used during training and load the dataset.

In [3]:
# Map component names from config to actual classes
# This is a bit manual; a more robust solution might use a factory or registration system.
all_components = {
    "pre_processing": {
        "StandardPreProcessing": StandardPreProcessing(n_categorical_levels=32) # Assuming this was the setting
    },
    "input_encoding": {
        "NoInputEncoder": NoInputEncoder(),
        "RecordLevelEmbed": RecordLevelEmbed(64), # Placeholder, params might differ
        "CategoricalFeatureEmbed": CategoricalFeatureEmbed(EmbedLayerType.Dense, 16), # Placeholder
    },
    "sequential_model": {
        "BasicTransformer": BasicTransformer(2, 128, n_heads=2), # Placeholder
        "GPTSmallTransformer": GPTSmallTransformer(),
        "BERTSmallTransformer": BERTSmallTransformer()
    },
    "classification_head": {
        "LastTokenClassificationHead": LastTokenClassificationHead(),
        "FlattenClassificationHead": FlattenClassificationHead(),
        "GlobalAveragePoolingClassificationHead": GlobalAveragePoolingClassificationHead(),
        "CLSTokenClassificationHead": CLSTokenClassificationHead(),
        "FeaturewiseEmbedding": FeaturewiseEmbedding(project=False), # Placeholder
    }
}

model_config = config['model_config']

# Recreate the FlowTransformer instance
ft = FlowTransformer(
    pre_processing=all_components["pre_processing"]["StandardPreProcessing"], # Assuming standard
    input_encoding=all_components["input_encoding"][model_config['input_encoding']],
    sequential_model=all_components["sequential_model"][model_config['sequential_model']],
    classification_head=all_components["classification_head"][model_config['classification_head']],
    params=FlowTransformerParameters(
        window_size=model_config['window_size'],
        mlp_layer_sizes=model_config['mlp_layer_sizes'],
        mlp_dropout=model_config['mlp_dropout']
    )
)

print("FlowTransformer instance recreated.")

# Load the dataset using the config information
dataset_config = config['dataset']
dataset_spec_map = {
    "unified_flow_format": NamedDatasetSpecifications.unified_flow_format,
    "nsl_kdd": NamedDatasetSpecifications.nsl_kdd,
    "CSE_CIC_IDS": NamedDatasetSpecifications.unified_flow_format # Add the missing entry
}

ft.load_dataset(
    dataset_config['name'],
    "/home/joeldan/dvcon_model/FlowTransformer_Pytorch_Imp/datasets.csv", # Use the correct path
    dataset_spec_map[dataset_config['name']],
    evaluation_dataset_sampling=EvaluationDatasetSampling[dataset_config['eval_method']],
    evaluation_percent=dataset_config['eval_percent']
)

print("\nDataset loaded and processed.")

FlowTransformer instance recreated.
Using cache file path: cache/CSE_CIC_IDS_0_QdLmZHuh8yOmlGcKBEkf7hepImY0_VHNk9ujbqtTXGSrgVayeqG486IQ0.feather
Reading directly from cache cache/CSE_CIC_IDS_0_QdLmZHuh8yOmlGcKBEkf7hepImY0_VHNk9ujbqtTXGSrgVayeqG486IQ0.feather...

Dataset loaded and processed.

Dataset loaded and processed.


## 3. Evaluate Model Performance

Now, let's evaluate the loaded model on the evaluation dataset.

In [4]:
from framework.enumerations import CategoricalFormat
from typing import List

# --- Logic to prepare evaluation data, adapted from FlowTransformer.evaluate ---

print("Preparing evaluation data...")

selectable_mask = np.zeros(len(ft.X), dtype=bool)
selectable_mask[ft.parameters.window_size:-ft.parameters.window_size] = True
train_mask = ft.training_mask

y_mask = ~(ft.y.astype('str') == str(ft.dataset_specification.benign_label))

indices_test:np.ndarray = np.argwhere(~train_mask & selectable_mask).reshape(-1)

def get_windows_for_indices(indices:np.ndarray) -> List[pd.DataFrame]:
    X_windows: List[pd.DataFrame] = []
    for i1 in indices:
        X_windows.append(ft.X.iloc[(i1 - ft.parameters.window_size) + 1:i1 + 1])
    return X_windows

feature_columns_map = {}
def samplewise_to_featurewise(X_windows: List[pd.DataFrame]) -> List[np.ndarray]:
    sequence_length = len(X_windows[0])
    combined_df = pd.concat(X_windows)
    featurewise_X = []
    
    if len(feature_columns_map) == 0:
        for feature in ft.model_input_spec.feature_names:
            if feature in ft.model_input_spec.numeric_feature_names or ft.model_input_spec.categorical_format == CategoricalFormat.Integers:
                feature_columns_map[feature] = feature
            else:
                feature_columns_map[feature] = [c for c in X_windows[0].columns if str(c).startswith(feature)]

    for feature in ft.model_input_spec.feature_names:
        feature_columns = feature_columns_map[feature]
        combined_values = combined_df[feature_columns].values
        
        reshaped_values = np.array([combined_values[i:i+sequence_length] for i in range(0, len(combined_values), sequence_length)])
        featurewise_X.append(reshaped_values)
        
    return featurewise_X

eval_X_windows = get_windows_for_indices(indices_test)
eval_X = samplewise_to_featurewise(eval_X_windows)
eval_y = y_mask[indices_test].astype(int)

print("Evaluation data prepared.")

# --- End of data preparation logic ---


# Evaluate the model
print("\nEvaluating model on the test dataset...")
loss, accuracy = model.evaluate(eval_X, eval_y, batch_size=config['training_config']['batch_size'])
print(f"\nEvaluation results:")
print(f"  - Loss: {loss:.4f}")
print(f"  - Accuracy: {accuracy:.4f}")

# Get predictions for more detailed analysis
print("\nGenerating predictions for classification report and confusion matrix...")
y_pred_probs = model.predict(eval_X, batch_size=config['training_config']['batch_size'])
y_pred = (y_pred_probs > 0.5).astype(int)

# Get true labels
y_true = eval_y

print("Done.")

Preparing evaluation data...
Evaluation data prepared.

Evaluating model on the test dataset...
Evaluation data prepared.

Evaluating model on the test dataset...


/home/joeldan/miniconda3/envs/flowtransformer/lib/python3.10/site-packages/keras/src/models/functional.py:225: UserWarning: The structure of `inputs` doesn't match the expected structure: ['input_MAX_IP_PKT_LEN', 'input_NUM_PKTS_128_TO_256_BYTES', 'input_NUM_PKTS_256_TO_512_BYTES', 'input_RETRANSMITTED_OUT_PKTS', 'input_LONGEST_FLOW_PKT', 'input_TCP_WIN_MAX_OUT', 'input_NUM_PKTS_1024_TO_1514_BYTES', 'input_DURATION_OUT', 'input_IN_BYTES', 'input_RETRANSMITTED_IN_PKTS', 'input_TCP_WIN_MAX_IN', 'input_RETRANSMITTED_OUT_BYTES', 'input_DURATION_IN', 'input_SRC_TO_DST_SECOND_BYTES', 'input_RETRANSMITTED_IN_BYTES', 'input_FLOW_DURATION_MILLISECONDS', 'input_MIN_IP_PKT_LEN', 'input_OUT_PKTS', 'input_NUM_PKTS_UP_TO_128_BYTES', 'input_NUM_PKTS_512_TO_1024_BYTES', 'input_SHORTEST_FLOW_PKT', 'input_IN_PKTS', 'input_DST_TO_SRC_AVG_THROUGHPUT', 'input_OUT_BYTES', 'input_MAX_TTL', 'input_DST_TO_SRC_SECOND_BYTES', 'input_MIN_TTL', 'input_SRC_TO_DST_AVG_THROUGHPUT', 'input_CLIENT_TCP_FLAGS', 'input_L4

26/39 ━━━━━━━━━━━━━━━━━━━━ 0s 7ms/step - binary_accuracy: 0.9899 - loss: 0.0681

I0000 00:00:1753199252.011230 1225353 device_compiler.h:188] Compiled cluster using XLA!  This line is logged at most once for the lifetime of the process.


39/39 ━━━━━━━━━━━━━━━━━━━━ 5s 6ms/step - binary_accuracy: 0.9906 - loss: 0.0638
39/39 ━━━━━━━━━━━━━━━━━━━━ 5s 6ms/step - binary_accuracy: 0.9906 - loss: 0.0638

Evaluation results:
  - Loss: 0.0537
  - Accuracy: 0.9924

Generating predictions for classification report and confusion matrix...

Evaluation results:
  - Loss: 0.0537
  - Accuracy: 0.9924

Generating predictions for classification report and confusion matrix...
39/39 ━━━━━━━━━━━━━━━━━━━━ 2s 7ms/step
39/39 ━━━━━━━━━━━━━━━━━━━━ 2s 7ms/step
Done.
Done.


## 4. Analyze and Visualize Results

Display the classification report and confusion matrix to get a deeper insight into the model's performance.

In [ ]:
# Classification Report
print("Classification Report:")
# Since this is a binary classification (Benign/Malicious), we can define the names directly.
target_names = ['Benign', 'Malicious']
print(classification_report(y_true, y_pred, target_names=target_names))

# Confusion Matrix
print("\nConfusion Matrix:")
cm = confusion_matrix(y_true, y_pred)
plt.figure(figsize=(8, 6))
sns.heatmap(cm, annot=True, fmt='d', cmap='Blues', xticklabels=target_names, yticklabels=target_names)
plt.title('Confusion Matrix')
plt.ylabel('Actual')
plt.xlabel('Predicted')
plt.show()

Classification Report:


AttributeError: 'FlowTransformer' object has no attribute 'n_classes'